In [141]:
import ldclient
from ldclient import Context
from ldclient.config import Config
from faker import Faker
from uuid import uuid4
from faker.providers import DynamicProvider 
import time
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json


In [142]:
sdk_key = 'sdk-c6b1251d-06e7-4efd-9486-2c51570cc5d3'
config = Config(sdk_key = sdk_key, http=ldclient.config.HTTPConfig(connect_timeout=5))
ldclient.set_config(config)
client = ldclient.get()

In [143]:
brands_provider  = DynamicProvider(
     provider_name="brand",
     elements=["Admiral", "Diamond", "Elephant", "Toothbrush", "Biscuit"],
)

product_provider = DynamicProvider(
    provider_name="product",
    elements=["Car", "Home", "Motorcycle", "Renters"],
)

price_provider = DynamicProvider(
    provider_name="example_in_cart_price",
    elements=[93.84, 143.73, 101.35, 86.02, 46.91, 125.62, 77.85, 99.68, 73.99, 148.79],
)


tier_provider = DynamicProvider(
    provider_name="tier",
    elements=["Bronze", "Silver", "Gold"],
)



fake = Faker()
fake.add_provider(brands_provider)
fake.add_provider(price_provider)
fake.add_provider(product_provider)
fake.add_provider(tier_provider)

# This is the start of some work on the Percentage Rollout stuff

In [144]:
num_records = 100
i = 0
controlProb = 0.3
treatmentProb = 0.35
useProb = 0
fakeUserId2 = []
user2Brand = [] 
user2Price = []
user2Product = []
user2Tier = []

while i <= num_records :
    fakeUserId2.append(fake.uuid4())
    user2Brand.append(fake.brand())
    user2Price.append(fake.example_in_cart_price())
    user2Product.append(fake.product())
    user2Tier.append(fake.tier())
    i = i+1

In [145]:
i = 0
run1 = []
run1_control = []
run1_treatment = []
run1_keys = []

while i <= num_records :

    user2_context = Context.builder(fakeUserId2[i]).kind("user").set("brand", user2Brand[i]).set("product", user2Product[i]).set("tier", user2Tier[i]).build()
    variation_detail = client.variation_detail("example-percentage-rollout-flag", user2_context, 'false')
    run1_keys.append(user2_context.key)
    if (variation_detail.value == "Control"):
        run1_control.append(user2_context.key)
    else:
        run1_treatment.append(user2_context.key)
    time.sleep(0.01)
    run1.append(variation_detail)
    i = i+1

# Here is where you should go and make a change to the percentage rollout

In [146]:
i = 0
run2 = []
run2_control = []
run2_treatment = []
run2_keys = []

while i <= num_records :

    user2_context = Context.builder(fakeUserId2[i]).kind("user").set("brand", user2Brand[i]).set("product", user2Product[i]).set("tier", user2Tier[i]).build()
    variation_detail = client.variation_detail("example-percentage-rollout-flag", user2_context, 'false')
    run2_keys.append(user2_context.key)
    if (variation_detail.value == "Control"):
        run2_control.append(user2_context.key)
    else:
        run2_treatment.append(user2_context.key)
    time.sleep(0.01)
    run2.append(variation_detail)
    i = i+1

In [147]:
i=0
mismatchCount = 0
while i <= num_records : 
   if (run1[i] != run2[i]):
      mismatchCount+=1
   i+=1   
print(mismatchCount)
#print("Run 1: Control: ", run1_control, "Treatment: ", run1_treatment)
#print("Run 2: Control: ", run2_control, "Treatment: ", run2_treatment)

23


In [148]:
def parse_string_to_dict(item):
    # Remove surrounding parentheses and split by commas
    clean_item = item.strip("()")
    key_value_pairs = clean_item.split(", ")
    
    # Convert key-value pairs into a dictionary
    parsed_dict = {}
    for pair in key_value_pairs:
        key, value = pair.split("=")  # Split by '='
        parsed_dict[key.strip()] = value.strip()
    
    return parsed_dict

In [149]:
run1_parsed_data = [parse_string_to_dict(str(item)) for item in run1]
run2_parsed_data = [parse_string_to_dict(str(item)) for item in run2]


In [150]:
run1_df = pd.DataFrame(run1_parsed_data)
run2_df = pd.DataFrame(run2_parsed_data)
keys_df = pd.DataFrame(run1_keys, columns=['key'])
run1_df['key'] = keys_df['key']
run2_df['key'] = keys_df['key']
result = pd.merge(run1_df, run2_df, on='key')

In [151]:
result[['key','value_x', 'value_y']]

,key,value_x,value_y
0,4aaf5bbb-e4f6-414f-b484-ee2b7bfbd90e,Control,Treatment
1,cbff163e-7260-4f28-9080-a0e515eaf184,Treatment,Treatment
2,9e1d5e01-d947-49fd-bdaf-ad0d6b1efba5,Control,Control
3,60712106-7353-4f69-a6eb-39788ef4eb1b,Control,Treatment
4,f833b033-6742-41df-bb22-a020ecb472c1,Control,Control
...,...,...,...
96,bf241f52-b719-4fdb-a826-503cda9b7963,Control,Control
97,164cc8bd-be89-40a7-a267-672eca46870d,Treatment,Treatment
98,b83e83b2-b7b4-41c4-8633-0fe0c39b40bc,Treatment,Treatment
99,ea38bebc-2907-4aea-97c8-7ea51a2e0459,Treatment,Treatment
